In [121]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import sherpa
import pandas as pd
import numpy as np
from tqdm import tqdm

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import time

import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from config import C_COMMON, C_GRID, C_SINGLE, FILENAME
from util import load_data, NeuralNetwork

from sklearn.linear_model import LinearRegression


# enable autoreload
%load_ext autoreload
%autoreload 2

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model

def define_model(
    input_dim=20,
    n_units=512,
    activation='selu',#selu
    learning_rate=0.00001,
    loss='mse',
    batch_size=64
):
    inputs = Input(shape=(input_dim))
    # x = Dense(units=n_units, activation=activation, kernel_regularizer='l1')(inputs)
    x = Dense(units=n_units, activation=activation)(inputs)
    x = Dropout(rate=0.5)(x)
    # x = Dense(units=n_units, activation=activation, kernel_regularizer='l1')(x)
    x = Dense(units=n_units, activation=activation)(inputs)
    x = Dropout(rate=0.5)(x)
    # x = Dense(units=n_units, activation=activation, kernel_regularizer='l1')(x)
    x = Dense(units=n_units, activation=activation)(inputs)
    x = Dropout(rate=0.5)(x)# serves as regularization
    outputs = Dense(units=1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[RootMeanSquaredError()]
    )
    return model, batch_size


In [28]:
columns = C_SINGLE
n_run = 100

df_train, _ = load_data(columns + C_COMMON, FILENAME)

In [34]:
# set up sherpa
parameters = [
    sherpa.Discrete('n_units', [256, 1024]),
    sherpa.Continuous('learning_rate', [0.00001, 0.01]),
    sherpa.Choice('batch_size', [64, 128, 192, 256, 512]),
    sherpa.Choice('loss', ['mse', 'mae'])
]
study = sherpa.Study(
    parameters=parameters,
    algorithm=sherpa.algorithms.RandomSearch(max_num_trials=n_run),
    lower_is_better=True
)

dfs = []
for i, trial in enumerate(study):
    # obtain hyperparameters
    params = {key: val for key, val in trial.parameters.items()}
    params['input_dim'] = len(columns)

    for skn in tqdm(df_train['skn'].unique()):
        model = NeuralNetwork(
            model_func=define_model,
            params=params,
            columns=columns
        )
        ret = model.cross_val_predict(df_train, skn)
        df = pd.DataFrame()
        df['trial_id'] = i,
        df['rmse'] = [ret['rmse']]
        df['mae'] = [ret['mae']]
        df['skn'] = [skn]
        for key, val in params.items():
            df[key] = [val]
        dfs.append(df)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://10.100.11.207:8887 if on a cluster or
http://localhost:8887 if running locally.
-------------------------------------------------------


 * Serving Flask app 'sherpa.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


100%|██████████| 24/24 [00:00<00:00, 338.10it/s]


# get the result

In [3]:
df_report = pd.read_csv('nn_report_500_single.csv')

In [4]:
df_groupby = df_report.groupby(by='trial_id').mean()

In [32]:
objective = 'rmse'
params = df_groupby[df_groupby[objective] == df_groupby[objective].min()][['n_units', 'learning_rate', 'batch_size', 'input_dim']].iloc[0].to_dict()
{key: val if key == 'learning_rate' else int(val) for key, val in params.items()}

{'n_units': 835,
 'learning_rate': 0.0002633633813255,
 'batch_size': 64,
 'input_dim': 16}

In [481]:
params = {}
idx = min_mse_index
params['input_dim'] = int(df_report[df_report['trial_id'] == idx].iloc[0]['input_dim'])
params['n_units'] = int(df_report[df_report['trial_id'] == idx].iloc[0]['n_units'])
params['learning_rate'] = float(df_report[df_report['trial_id'] == idx].iloc[0]['learning_rate'])
params['loss'] = df_report[df_report['trial_id'] == idx].iloc[0]['loss']
batch_size = df_report[df_report['trial_id'] == idx].iloc[0]['batch_size']

In [483]:
df_report[df_report['trial_id'] == idx]

,Unnamed: 0,input_dim,n_units,learning_rate,loss,skn,batch_size,mae,rmse,trial_id
1776,0,16,362.231815,0.000695,mse,54.0,128,3.433046,5.296056,74
1777,0,16,362.231815,0.000695,mse,79.0,128,3.886207,5.973462,74
1778,0,16,362.231815,0.000695,mse,338.0,128,2.544070,4.762007,74
1779,0,16,362.231815,0.000695,mse,250.0,128,1.394395,2.049286,74
1780,0,16,362.231815,0.000695,mse,267.0,128,1.505138,2.224552,74
1781,0,16,362.231815,0.000695,mse,296.1,128,0.799617,1.681203,74
1782,0,16,362.231815,0.000695,mse,311.0,128,0.796940,1.520546,74
1783,0,16,362.231815,0.000695,mse,396.0,128,1.051577,1.784216,74
1784,0,16,362.231815,0.000695,mse,400.0,128,1.138457,1.805463,74
1785,0,16,362.231815,0.000695,mse,406.0,128,1.241432,1.891523,74


In [490]:
list_ypred = []
list_ytrue = []
for fold in range(5):
    x_train, x_test, y_train, y_test= prepare_dataset(df_inner_split, skn=skn, inner_fold=0)
    linear_regression = LinearRegression()
    linear_regression.fit(x_train, y_train)
    yhat = linear_regression.predict(x_test)
    list_ytrue.append(y_test)
    list_ypred.append(yhat)
mean_absolute_error(list_ytrue, list_ypred)

4.524202419975154